# Mapping, merging, and finding Barcode-TatSL8 Linkage
#### Written by Ryan V. Moriarty, 10/8/21

This jupyter notebook was modified from its original verison, written by Athena E. Golfinos, and available at: https://github.com/agolfinos/SIV_Transmission_Study. 

This pipeline takes fastq files that have been demultiplexed, and merges them using bbmerge, and maps them to the reference strain SIVmac239M (SIVmac239M is derived from SIVmac239, found with accession M22362 on Genbank) using bbmap. BBtools is available on SourceForge (https://sourceforge.net/projects/bbmap/). Once mapped, the barcode and TatSL8 nucleotide sequences are extracted from each read, and written to a .csv file for further processing using R. 

# Importing Packages

In [ ]:
import sys
import os
import pandas as pd
from Bio import SeqIO, Seq
import subprocess
import glob

# Defining Variables

In [ ]:
appRoot = "/Users/ryanmoriarty/Documents/SIV_transmission_reanalysis_info/SIV_transmission_reanalysis/6196_files_linkage"
subfolders = ["set1", "set2"]
ref = appRoot + "/ref/M33262_barcode.fasta"

os.chdir(appRoot)

## Merge paired Fastq files for each unique sample

In [ ]:
for f in subfolders:
    # Begin by switching to the subfolder 
    os.chdir(appRoot + "/" + f)
    
    # Identify each unique sample 
    sample_full_name = glob.glob("*.fastq")
    sample_uniques = list(set([s[:s.find("_R")] for s in sample_full_name]))
    
    # Now that we have a list of unique samples present in that folder, find their R1 and R2, merge them using 
    # bbmerge, and map them using bbmap. 
    for s in sample_uniques:
        r1 = os.getcwd() + "/" + glob.glob(s+"*")[0]
        r2 = os.getcwd() + "/" + glob.glob(s+"*")[1]
        merged = s + ".merged.fastq"
        
        merge_cmd = [appRoot + '/bbmap/bbmerge.sh', "in=" + r1, "in2=" + r2, "out=" + merged, "qtrim=t",
            "trimq=20"]
        subprocess.call(merge_cmd)

## Map merged Fastq files to the barcoded SIVmac239M reference

In [ ]:
for f in subfolders:
    # Begin by switching to the subfolder 
    os.chdir(appRoot + "/" + f)
    # Because there may be other files in your folder, make sure you are only mapping the ones merged in the previous 
    # kernel 
    merged_files = glob.glob("*.merged.fastq")
    for m in merged_files:
        # the name of your mapped files will come from the name of your merged, minus the 6 characters of ".fastq"
        mapped = m[:-6] + ".mapped.fastq"
        map_cmd = [appRoot + '/bbmap/bbmap.sh', "ref=" + ref, "in=" + m,  
          "outm=" + mapped]
        subprocess.call(map_cmd)

.
# Extracting Tat SL8 to Barcode Linkages

#### This section is from the TatSL8_to_barcode.ipynb script found on Athena Golfinos's github, accessible through this link: https://github.com/agolfinos/SIV_Transmission_Study

In [ ]:
allLinkageData = []
#for the files in our out directory
# If redoing analysis from this paper, split the two animals (r04103 and r10001) into two separate folders, as the
# barcode is found on the reverse read in r04103. 
for file in os.listdir(appRoot): 
    allAmplicons = []
    #ONLY take the files that end with fastq, there are also some stats files and hidden files that we don't want
    if file.endswith(".fastq"): 
        print(file)
        #using the Biopython parse we can read our fastq input
        fastq_parser = SeqIO.parse(appRoot +"/"+ file, "fastq")
        for seq_record in fastq_parser: 
            #Take the current sequence
            sequence = str(seq_record.seq)
            allAmplicons.append(sequence)
            #only finding the amplicons that are long enough to be real      
    for amplicon in allAmplicons: 
        if (len(amplicon)) >= 451: 
            #we are making a list to append our barcode/SL8 variant pair to
            link = []
            #this will give us our animal name in one column
            link.append(file[0:6])
            #this will give us our timepoint in one column
            link.append(file.split("_")[1])
            #this will give us our rep number in one column
            link.append(file.split("_")[2][3:4])
            
            #this is necessary for r04103
            #amplicon5 = Seq.reverse_complement(amplicon[1:])
            
            #pulling out the barcode from r04103
            #tag = str(amplicon5[73:107])
            
            #pulling out the barcode from r10001
            tag = str((amplicon[1:])[72:106])
            
            link.append(tag)
            
            #now we are pulling out the Tat SL8 sequence for r04103
            #SL8 = str(amplicon5[339:363])
            
            #now we are pulling out the Tat SL8 sequence for r10001
            SL8 = str((amplicon[1:])[338:362])
            
            link.append(SL8)
            #now we want to translate SL8
            SL8AA = Seq.translate(SL8)
            #now we are adding the amino acid sequence of SL8 as the second part of the list
            link.append(SL8AA)
            allLinkageData.append(link)

df = pd.DataFrame(allLinkageData, columns=["Animal ID", "Time Point", 
                                           "Rep Number", "SIVmac239M Barcode", 
                                           "Tat SL8 Nucleotide Sequence",
                                           "Tat SL8 Amino Acid Sequence"])
linkage_df = df.groupby(["Animal ID", "Time Point", "Rep Number", 
                          "SIVmac239M Barcode", 
                          "Tat SL8 Nucleotide Sequence", 
                          "Tat SL8 Amino Acid Sequence"]).size().reset_index().rename(columns={0:'count'})
linked_sorted_df = linkage_df.sort_values(by=['count'], ascending=False)
print(linked_sorted_df)
linked_sorted_df.to_csv(appRoot+ "/" + "r04103_IR_TatSL8toBarcodeLinkage_sep.csv")           